In [236]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [237]:
base = pd.read_csv('./data/ObesityDataSet_raw_and_data_sinthetic.csv')

data_frame = pd.DataFrame(base)
data_frame.rename({'NObeyesdad': 'target'}, axis=1, inplace=True)
data_frame.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,target
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [238]:
y = data_frame['target']

In [239]:
nominal_columns = ['MTRANS']
data = pd.get_dummies(data_frame, prefix=nominal_columns, columns=nominal_columns)
all_columns_name = data_frame.columns.to_list()

In [250]:
ordinal_columns = ['CAEC', 'CALC']
ordinal_encode = OrdinalEncoder()
ordinal_data = ordinal_encode.fit_transform(data[ordinal_columns])


array([[2.000e+00, 3.000e+00, 5.490e+02],
       [2.000e+00, 2.000e+00, 1.267e+03],
       [2.000e+00, 1.000e+00, 5.490e+02],
       ...,
       [2.000e+00, 2.000e+00, 6.150e+02],
       [2.000e+00, 2.000e+00, 1.181e+03],
       [2.000e+00, 2.000e+00, 1.189e+03]])

In [241]:

obesity_df = pd.DataFrame(data, columns=all_columns_name)
obesity_df

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,target
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,NaN,Normal_Weight
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,NaN,Normal_Weight
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,NaN,Normal_Weight
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,NaN,Overweight_Level_I
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,NaN,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,NaN,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,NaN,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,NaN,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,NaN,Obesity_Type_III


In [242]:
X = obesity_df.drop('target', axis=1)
X_norm = MinMaxScaler().fit_transform(X)
X_norm

ValueError: could not convert string to float: 'Female'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=1/3, random_state=1057)

knn_obesity = KNeighborsClassifier(n_neighbors=3)
knn_obesity.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=2)

In [ ]:
accuracy_score(y_test, knn_obesity.predict(X_test))

0.8167613636363636